<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install ta
!pip install yfinance==0.2.59

  Installing build dependencies ... -

 \

 done


  Getting requirements to build wheel ... -

 done


...

 done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29482 sha256=3bb4b74ec72a96b87194d7894fd7dd421afd32dc67731ceedc62dd7894ee67a4
  Stored in directory: /home/runner/.cache/pip/wheels/e3/3a/ee/4955a26c90a4b7deb6d725dc8ec7b8604a7aef44e43a2e8af7
Successfully built ta


  Attempting uninstall: protobuf


    Found existing installation: protobuf 6.32.1
    Uninstalling protobuf-6.32.1:
      Successfully uninstalled protobuf-6.32.1


  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.66
    Uninstalling yfinance-0.2.66:
      Successfully uninstalled yfinance-0.2.66
   ━━━━━━━━━━━━━━━━━━━━╺━━━━━━━━━━━━━━━━━━━ 1/2 [yfinance]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-10-07 11:10:40
-------------------
qualified stocks: 87
with latest results: 27
still star stocks: 16
-------------------
Initial Investment:  1.24 C
CY Investment:  1.51 C
Reserve:  57.44 K
Current:  1.41 C
-------------------
Today PnL: 15.88 K (0.11%)
Current PnL: -20.41 L (-13.55%)
CY Booked + Current PnL: -8.97 L (-5.95%)
-------------------
Total profit:  1.51 L
Total loss:  -21.92 L
-------------------
Total Booked + Current PnL: 18.01 L (14.56%)
Total Booked PnL: 38.42 L (31.06%)
Curr Year Booked PnL: 11.44 L (8.1%)
Prev Year Booked PnL: 26.98 L (21.81%)
Est FTT:  2.29 C
Est FTT PnL: 87.92 L (62.27%)
Deployed:  1.24 C
Current:  1.41 C
CAGR/XIRR %: 8.02%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'FTT', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%', 'Current', 'Current P/L', 'FTT Amt', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
50,NATIONALUM,244.55,-42.00,63.0,H-MC,13.74,115381.0,14877.0,14226.0,0.25,14.80,12.33,28.96,79.0,1.05,0.82,53.80,MH,ATH,METALS
14,BLUESTARCO,2080.00,11.47,54.0,H-MC,3.01,192500.0,27830.0,15496.0,1.70,16.90,8.05,26.31,89.0,1.80,1.37,26.23,X40N,NTT,AC
77,TTKPRESTIG,770.00,93.17,38.0,M-SC,1.35,83211.0,-17566.0,17657.0,-0.03,-17.43,21.22,0.09,245.0,-0.99,0.59,9.32,OX40N,NTT,DURABLES
19,CIPLA,1662.74,-22.18,45.0,H-LC,3.91,207911.0,3411.0,19876.0,0.30,1.67,9.56,11.39,10.0,0.17,1.48,10.77,X40N,ATH,PHARMA
57,RELIANCE,1533.00,-13.66,50.0,H-LC,3.33,215732.0,5366.0,23407.0,0.57,2.55,10.85,13.68,37.0,0.23,1.53,19.50,XY25,NTT,REFINERIES


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
4,ANGELONE,3033.0,5.10,50.0,H-SC,11.27,185746.0,-5260.0,62968.0,2.90,-2.75,33.90,30.21,157.0,-0.08,1.32,17.55,X40N,NTT,FINANCE
68,STARHEALTH,761.0,14.40,65.0,H-SC,5.78,254810.0,-26313.0,159180.0,2.38,-9.36,62.47,47.26,144.0,-0.17,1.81,36.52,XY24,NTT,INSURANCE
85,WHIRLPOOL,2270.0,-46.35,35.0,M-SC,1.09,90608.0,-890.0,79644.0,2.10,-0.97,87.90,86.07,223.0,-0.01,0.64,31.58,XR,NTT,DURABLES
40,INFY,2275.0,-19.24,49.0,H-LC,8.13,314388.0,1500.0,170178.0,2.03,0.48,54.13,54.87,3.0,0.01,2.24,7.16,X40,BTT,IT
55,RAJOOENG,143.1,-39.13,47.0,H-SC,16.24,92300.0,-10200.0,50802.0,1.94,-9.95,55.04,39.61,114.0,-0.20,0.66,12.73,AR,ATH,MISC


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
64,SHALBY,327.00,1218.98,63.0,M-SC,20.82,179279.0,-1864.0,47330.0,-2.38,-1.03,26.40,25.10,235.0,-0.04,1.28,43.67,XY24,NTT,HEALTHCARE
38,INDIGOPNTS,1408.00,100.89,36.0,M-SC,2.11,131849.0,-42710.0,42745.0,-1.66,-24.47,32.42,0.02,221.0,-1.00,0.94,14.69,OX40N,NTT,PAINTS
69,SURYODAY,240.00,62.28,63.0,H-SC,6.67,149083.0,-29988.0,95711.0,-1.38,-16.75,64.20,36.71,135.0,-0.31,1.06,47.92,XR,NTT,BANKS
82,VALIANTORG,838.00,-301.50,38.0,H-SC,2.79,112081.0,-51524.0,172840.0,-1.24,-31.49,154.21,74.15,139.0,-0.30,0.80,43.67,XR,NTT,CHEMICALS
71,TANLA,1963.11,-19.92,54.0,H-SC,19.32,215179.0,-55582.0,385536.0,-1.22,-20.53,179.17,121.86,133.0,-0.14,1.53,68.33,AR,ATH,IT


In [12]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
85,WHIRLPOOL,2270.0,-46.35,35.0,M-SC,1.09,90608.0,-890.0,79644.0,2.10,-0.97,87.90,86.07,223.0,-0.01,0.64,31.58,XR,NTT,DURABLES
35,ICICIPRULI,790.0,-20.77,51.0,H-MC,2.04,136391.0,603.0,42145.0,0.51,0.44,30.90,31.48,107.0,0.01,0.97,12.62,X40,ATH,INSURANCE
86,WIPRO,420.0,-14.70,44.0,M-LC,5.71,150294.0,-651.0,110105.0,0.12,-0.43,73.26,72.51,53.0,-0.01,1.07,5.57,XR,NTT,IT


In [13]:
# OX40N stocks to exit
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
78,UJJIVANSFB,60.0,107.70,66.0,H-SC,13.53,131544.0,-10935.0,30452.0,-0.35,-7.67,23.15,13.70,163.0,-0.36,0.94,55.26,OX40N,NTT,BANKS
45,KANSAINER,340.0,-67.42,56.0,H-SC,2.11,225000.0,-44667.0,81000.0,-0.39,-16.56,36.00,13.47,138.0,-0.55,1.60,14.41,XY24,NTT,PAINTS
77,TTKPRESTIG,770.0,93.17,38.0,M-SC,1.35,83211.0,-17566.0,17657.0,-0.03,-17.43,21.22,0.09,245.0,-0.99,0.59,9.32,OX40N,NTT,DURABLES
18,CERA,9475.0,-22.43,40.0,H-SC,1.51,141254.0,-34649.0,76673.0,0.30,-19.70,54.28,23.89,149.0,-0.45,1.00,21.99,OX40N,NTT,CERAMICS
72,TATAELXSI,9161.0,-25.05,40.0,H-MC,6.27,101346.0,-30489.0,72716.0,-1.11,-23.13,71.75,32.03,98.0,-0.42,0.72,13.52,OX40N,NTT,IT


In [14]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [15]:
# top 5 to monitor for exit
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
60,SAIL,228.00,44.35,51.0,M-MC,12.35,227425.0,2463.0,165202.0,-0.41,1.09,72.64,74.53,192.0,0.01,1.62,33.70,XY24,BTT,STEEL
32,HINDZINC,730.22,29.80,65.0,M-LC,6.19,211845.0,6769.0,105795.0,-0.92,3.30,49.94,54.89,52.0,0.06,1.51,28.19,X5K,ATH,METALS
14,BLUESTARCO,2080.00,11.47,54.0,H-MC,3.01,192500.0,27830.0,15496.0,1.70,16.90,8.05,26.31,89.0,1.80,1.37,26.23,X40N,NTT,AC
84,VOLTAS,1530.00,-1.92,49.0,H-MC,1.43,205875.0,14133.0,23634.0,0.42,7.37,11.48,19.69,99.0,0.60,1.46,14.67,XY25,NTT,AC
17,CAMS,5211.76,-4.72,51.0,H-SC,1.79,108416.0,6412.0,37512.0,1.22,6.29,34.60,43.06,122.0,0.17,0.77,24.48,X40N,ATH,MISC


In [16]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
50,NATIONALUM,244.55,-42.00,63.0,H-MC,13.74,115381.0,14877.0,14226.0,0.25,14.80,12.33,28.96,79.0,1.05,0.82,53.80,MH,ATH,METALS
11,BANKINDIA,190.00,-24.22,71.0,H-MC,10.85,195141.0,15333.0,97083.0,0.67,8.53,49.75,62.52,88.0,0.16,1.39,45.52,XR,NTT,BANKS
32,HINDZINC,730.22,29.80,65.0,M-LC,6.19,211845.0,6769.0,105795.0,-0.92,3.30,49.94,54.89,52.0,0.06,1.51,28.19,X5K,ATH,METALS
37,INDIAMART,4810.62,-54.83,36.0,H-SC,9.35,126092.0,2756.0,128866.0,0.13,2.23,102.20,106.72,119.0,0.02,0.90,26.14,AR,ATH,MISC
85,WHIRLPOOL,2270.00,-46.35,35.0,M-SC,1.09,90608.0,-890.0,79644.0,2.10,-0.97,87.90,86.07,223.0,-0.01,0.64,31.58,XR,NTT,DURABLES


In [17]:
# top 5 to accumulate based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
83,VBL,671.64,-22.30,30.0,H-LC,9.96,279986.0,-35856.0,148533.0,-0.02,-11.35,53.05,35.67,5.0,-0.24,1.99,0.93,X40N,ATH,FMCG
5,ASIANPAINT,3465.66,-17.77,35.0,H-LC,5.77,207557.0,-44211.0,97427.0,0.16,-17.56,46.94,21.13,27.0,-0.45,1.48,11.64,X40,ATH,PAINTS
42,ITC,452.00,-39.19,42.0,H-LC,2.15,195991.0,-4147.0,25028.0,0.01,-2.07,12.77,10.44,4.0,-0.17,1.39,3.38,X40,NTT,FMCG
31,HINDUNILVR,2922.00,-9.85,43.0,H-LC,8.02,252920.0,-4143.0,39278.0,-0.50,-1.61,15.53,13.67,24.0,-0.11,1.80,18.47,XY25,NTT,FMCG
74,TCS,4406.34,-29.97,43.0,H-LC,12.19,278522.0,-67434.0,135668.0,-0.85,-19.49,48.71,19.73,1.0,-0.50,1.98,2.58,X40,ATH,IT


In [18]:
# top 5 to accumulate based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
65,SIEMENS,4671.5,0.17,54.0,H-LC,1.98,161005.0,-25090.0,72565.0,-0.97,-13.48,45.07,25.51,15.0,-0.35,1.15,18.65,AR,ATH,ELECTRICAL
42,ITC,452.0,-39.19,42.0,H-LC,2.15,195991.0,-4147.0,25028.0,0.01,-2.07,12.77,10.44,4.0,-0.17,1.39,3.38,X40,NTT,FMCG
48,LTIM,7230.2,-4.34,54.0,H-LC,2.36,237172.0,-8979.0,88181.0,-0.07,-3.65,37.18,32.18,16.0,-0.10,1.69,31.08,X200,ATH,IT
51,NESTLEIND,1377.0,-10.26,51.0,H-LC,3.05,276003.0,10577.0,46203.0,-0.19,3.98,16.74,21.40,11.0,0.23,1.96,10.89,XY25,NTT,FMCG
57,RELIANCE,1533.0,-13.66,50.0,H-LC,3.33,215732.0,5366.0,23407.0,0.57,2.55,10.85,13.68,37.0,0.23,1.53,19.50,XY25,NTT,REFINERIES


In [19]:
# top 5 to accumulate based on lowest Gained%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'Gained%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
83,VBL,671.64,-22.30,30.0,H-LC,9.96,279986.0,-35856.0,148533.0,-0.02,-11.35,53.05,35.67,5.0,-0.24,1.99,0.93,X40N,ATH,FMCG
74,TCS,4406.34,-29.97,43.0,H-LC,12.19,278522.0,-67434.0,135668.0,-0.85,-19.49,48.71,19.73,1.0,-0.50,1.98,2.58,X40,ATH,IT
42,ITC,452.00,-39.19,42.0,H-LC,2.15,195991.0,-4147.0,25028.0,0.01,-2.07,12.77,10.44,4.0,-0.17,1.39,3.38,X40,NTT,FMCG
80,UNITDSPR,1644.00,-11.37,56.0,H-LC,6.88,232252.0,-1892.0,50515.0,-0.79,-0.81,21.75,20.77,86.0,-0.04,1.65,6.05,X40N,NTT,BREWERIES
30,HCLTECH,1937.36,-1.51,49.0,H-LC,9.87,224808.0,-17116.0,79357.0,1.00,-7.08,35.30,25.73,8.0,-0.22,1.60,6.23,X40,ATH,IT


In [20]:
# top 5 to accumulate based on lowest CurrAlloc%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
65,SIEMENS,4671.50,0.17,54.0,H-LC,1.98,161005.0,-25090.0,72565.0,-0.97,-13.48,45.07,25.51,15.0,-0.35,1.15,18.65,AR,ATH,ELECTRICAL
42,ITC,452.00,-39.19,42.0,H-LC,2.15,195991.0,-4147.0,25028.0,0.01,-2.07,12.77,10.44,4.0,-0.17,1.39,3.38,X40,NTT,FMCG
19,CIPLA,1662.74,-22.18,45.0,H-LC,3.91,207911.0,3411.0,19876.0,0.30,1.67,9.56,11.39,10.0,0.17,1.48,10.77,X40N,ATH,PHARMA
5,ASIANPAINT,3465.66,-17.77,35.0,H-LC,5.77,207557.0,-44211.0,97427.0,0.16,-17.56,46.94,21.13,27.0,-0.45,1.48,11.64,X40,ATH,PAINTS
57,RELIANCE,1533.00,-13.66,50.0,H-LC,3.33,215732.0,5366.0,23407.0,0.57,2.55,10.85,13.68,37.0,0.23,1.53,19.50,XY25,NTT,REFINERIES


In [21]:
# top 5 to accumulate based on CumlRnk
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
74,TCS,4406.34,-29.97,43.0,H-LC,12.19,278522.0,-67434.0,135668.0,-0.85,-19.49,48.71,19.73,1.0,-0.50,1.98,2.58,X40,ATH,IT
40,INFY,2275.00,-19.24,49.0,H-LC,8.13,314388.0,1500.0,170178.0,2.03,0.48,54.13,54.87,3.0,0.01,2.24,7.16,X40,BTT,IT
42,ITC,452.00,-39.19,42.0,H-LC,2.15,195991.0,-4147.0,25028.0,0.01,-2.07,12.77,10.44,4.0,-0.17,1.39,3.38,X40,NTT,FMCG
83,VBL,671.64,-22.30,30.0,H-LC,9.96,279986.0,-35856.0,148533.0,-0.02,-11.35,53.05,35.67,5.0,-0.24,1.99,0.93,X40N,ATH,FMCG
1,ABB,7934.00,-39.18,56.0,H-LC,7.49,254160.0,-7459.0,126673.0,1.48,-2.85,49.84,45.57,7.0,-0.06,1.81,8.15,AR,NTT,ELECTRICAL


In [22]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
21,COFFEEDAY,80.00,-45.67,44.0,L-SC,31.61,79008.0,-34541.0,74591.0,-0.19,-30.42,94.41,35.27,268.0,-0.46,0.56,93.37,XR,NTT,HOTELS
49,MASFIN,398.61,-19.15,45.0,H-SC,11.45,91800.0,-6180.0,27779.0,-0.70,-6.31,30.26,22.05,152.0,-0.22,0.65,33.44,XR,ATH,FINANCE
50,NATIONALUM,244.55,-42.00,63.0,H-MC,13.74,115381.0,14877.0,14226.0,0.25,14.80,12.33,28.96,79.0,1.05,0.82,53.80,MH,ATH,METALS
78,UJJIVANSFB,60.00,107.70,66.0,H-SC,13.53,131544.0,-10935.0,30452.0,-0.35,-7.67,23.15,13.70,163.0,-0.36,0.94,55.26,OX40N,NTT,BANKS
61,SAMMAANCAP,326.00,-184.09,68.0,M-SC,12.55,145935.0,-3285.0,147467.0,1.38,-2.20,101.05,96.62,208.0,-0.02,1.04,57.93,XY25,NTT,FINANCE


In [23]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
61,SAMMAANCAP,326.00,-184.09,68.0,M-SC,12.55,145935.0,-3285.0,147467.0,1.38,-2.20,101.05,96.62,208.0,-0.02,1.04,57.93,XY25,NTT,FINANCE
69,SURYODAY,240.00,62.28,63.0,H-SC,6.67,149083.0,-29988.0,95711.0,-1.38,-16.75,64.20,36.71,135.0,-0.31,1.06,47.92,XR,NTT,BANKS
11,BANKINDIA,190.00,-24.22,71.0,H-MC,10.85,195141.0,15333.0,97083.0,0.67,8.53,49.75,62.52,88.0,0.16,1.39,45.52,XR,NTT,BANKS
32,HINDZINC,730.22,29.80,65.0,M-LC,6.19,211845.0,6769.0,105795.0,-0.92,3.30,49.94,54.89,52.0,0.06,1.51,28.19,X5K,ATH,METALS
26,GREENPANEL,537.00,229.46,58.0,M-SC,3.65,146986.0,-36092.0,115075.0,-0.33,-19.71,78.29,43.14,230.0,-0.31,1.05,38.26,XY24,NTT,MISC


In [24]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,19.12
1,25,43.28
2,50,74.97


In [25]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    44.07
LC    32.20
MC    23.74
Name: CurrAlloc%, dtype: float64

In [26]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     30.88
X40      14.60
X40N     11.22
XR       11.01
XY25     10.40
AR        8.23
OX40N     7.72
X200      1.69
MH        1.60
X5K       1.51
SR        1.15
Name: CurrAlloc%, dtype: float64

In [27]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-SC    29.00
H-LC    25.66
H-MC    20.51
M-SC    13.65
M-LC     5.52
M-MC     2.93
L-SC     1.42
L-LC     1.02
L-MC     0.30
Name: CurrAlloc%, dtype: float64

In [28]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,13.75,-6.14,41.12
IT,12.64,-18.43,82.57
FINANCE,10.49,-14.35,69.57
MISC,6.97,-17.50,82.61
BANKS,6.37,-11.39,69.98
PAINTS,5.56,-18.27,35.90
ELECTRICAL,5.38,-8.85,47.04
INSURANCE,3.80,-3.45,42.12
AC,3.52,2.24,14.13


In [29]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,3230359.0,22
XR,1363286.0,14
AR,1174698.0,9
X40,787529.0,10
X40N,656644.0,9
OX40N,562058.0,10
XY25,474506.0,7
SR,260524.0,2
X5K,105795.0,1


In [30]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,3714728.0,29
M-SC,1511962.0,17
H-MC,1269141.0,15
H-LC,1187782.0,15
M-LC,374153.0,4
M-MC,371660.0,2
L-SC,261605.0,3
L-MC,59907.0,1
L-LC,40764.0,1


In [31]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
H-SC       XY24      1211552.0      6
M-SC       XY24       853876.0      7
H-SC       AR         807395.0      5
           XR         795800.0      7
H-MC       XY24       570125.0      4
H-LC       X40        507658.0      5
M-MC       XY24       371660.0      2
H-SC       X40N       307424.0      4
           SR         260524.0      2
           OX40N      258137.0      4
H-LC       X40N       218924.0      3
H-MC       X40        207308.0      4
H-LC       AR         199238.0      2
H-MC       XY25       177387.0      2
L-SC       XR         172431.0      2
M-SC       AR         168065.0      2
M-LC       XY24       158253.0      2
M-SC       XY25       147467.0      1
           OX40N      142031.0      4
H-MC       X40N       130296.0      2
M-SC       XR         127960.0      2
M-LC       XR         110105.0      1
H-LC       XY25       108888.0      3
M-LC       X5K        105795.0      1
H-MC       XR          97083.0      1
L-SC       OX40N       89174.0      1
H-LC       X200        88181.0      1
H-SC       MH          73896.0      1
H-MC       OX40N       72716.0      1
M-SC       X40         72563.0      1
H-LC       XY24        64893.0      1
L-MC       XR          59907.0      1
L-LC       XY25        40764.0      1
H-MC       MH          14226.0      1

In [32]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 35.0 seconds
